# Lithology classification

## Data

This notebook uses data from the [Upper Condamine Catchment](http://www.bom.gov.au/qld/flood/brochures/condamine_balonne/map_upper.shtml) in the state of Queensland. The data is sourced from personal communication as a project output. It may be shared publicly and downloadable from this sample notebook in the future.

![Upper Condamine catchment formations](img/Upper_Condamine_formations.png "Upper Condamine catchment formations")

(Figure from [this paper](https://www.researchgate.net/figure/Upper-Condamine-catchment-Queensland-Australia-The-Marburg-Subgroup-consists-of_fig1_283184727))

## Status

As of May 2019 this present document is an output from exploratory work done during an internship by [Sudhir Gupta](https://github.com/Sudhir22).


## Purpose

This notebook compares the performance of two techniques for semi-automated classification . It also summarise work using ontologies for classification for cases where we do not have reliable training sets.


## Importing python packages

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd
import pickle

In [ ]:
# Only True for co-dev of ela from this use case:
ela_from_source = False
ela_from_source = True

In [ ]:
if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\Users\SUD011\Documents\pyela-sudhir'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/github_jm/pyela')
    pkg_src_dir = root_src_dir
    sys.path.append(pkg_src_dir)

from ela.textproc import *
from ela.utils import *
from ela.classification import *
from ela.visual import *

In [ ]:
import striplog
from striplog import Lexicon

In [ ]:
if ('ELA_DATA' in os.environ):
    data_path = os.environ['ELA_DATA']
elif sys.platform == 'win32':
    data_path = r'C:\data\Lithology'
else:
    username = os.environ['USER']
    data_path = os.path.join('/home', username, 'data', 'Lithology')

condamine_litho_dir = os.path.join(data_path,'Condamine')
condamine_litho_xl = os.path.join(condamine_litho_dir, 'MASTER_CONDAMINE_Interpretation_all_combined_Jan2017.xlsx')
condamine_litho_pkl = os.path.join(condamine_litho_dir, 'MASTER_CONDAMINE_Interpretation_all_combined_Jan2017.pkl')

# Initial data exploration

This first section deliberately covers some of the data discovery process for didactic purposes.

## Useful resources

Note that some of the terms used for classification are triangulated with [this description](https://www.bioregionalassessments.gov.au/assessments/21-22-data-analysis-clarence-moreton-bioregion/21211-lithological-and-stratigraphic-data)

We cache the data in a pickle as it is faster to re-read:


In [ ]:
if not os.path.exists(condamine_litho_pkl):
    train_data=pd.read_excel(condamine_litho_xl)
    with open(condamine_litho_pkl, 'wb') as handle:
        pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(condamine_litho_pkl, 'rb') as handle:
        train_data = pickle.load(handle)

In [ ]:
train_data.head()

In [ ]:
# train_data_unprocessed = train_data.copy()

In [ ]:
df = train_data
LITHO_DESC_COL='Lithology_original'
REGEX_LITHO_CLASS_COL='Regex_lithoclass'

LITHO_CLASS_COL = 'Simplified_lithology'

In [ ]:
set(train_data[LITHO_CLASS_COL].values)

We massage the column of simplified lithologies, resulting from a manual classification

In [ ]:
# This is not obvious from inspection of the the pandas  data frame, but there appears to be NaNs that cause headaches later on.
vv = [x for x in df[LITHO_DESC_COL].values if not type(x) is str]

In [ ]:
vv = [type(x) is not str for x in df[LITHO_DESC_COL].values]

In [ ]:
df.loc[np.array(vv)].head()

In [ ]:
train_data[LITHO_CLASS_COL] = train_data[LITHO_CLASS_COL].replace(np.nan,'',regex=True)
train_data[LITHO_CLASS_COL] = train_data[LITHO_CLASS_COL].str.lower()
train_data[LITHO_DESC_COL] = train_data[LITHO_DESC_COL].replace(np.nan,'',regex=True)
train_data[LITHO_DESC_COL] = train_data[LITHO_DESC_COL].str.lower()

In [ ]:
%%time

# Not obvious upfront but noticed that there are terms where dots and slashes prevent tokenization and lithology term detection. 
train_data[LITHO_DESC_COL] = v_replace_punctuations(train_data[LITHO_DESC_COL].values)

In [ ]:
train_data.head()

In [ ]:
token_freq(train_data[LITHO_CLASS_COL].values, 50)

Given the low frequency of some of the classes, we remap them to one of the three main classes:

In [ ]:
train_data[LITHO_CLASS_COL] = train_data[LITHO_CLASS_COL].replace('granite|granodiorite|diorite|basement','bedrock',regex=True)
train_data[LITHO_CLASS_COL] = train_data[LITHO_CLASS_COL].replace('gravel','alluvium',regex=False)
train_data[LITHO_CLASS_COL] = train_data[LITHO_CLASS_COL].replace('wrong_location|weathering_horizon|tertiary','unknown',regex=True)

In [ ]:
token_freq(train_data[LITHO_CLASS_COL].values, 50)

In [ ]:
descs = df[LITHO_DESC_COL]
descs = descs.reset_index()
descs = descs[LITHO_DESC_COL]
descs.head()

In [ ]:
lex = Lexicon.default()

In [ ]:
%%time
expanded_descs = descs.apply(lex.expand_abbreviations)
y = expanded_descs.values

In [ ]:
train_data[LITHO_DESC_COL] = expanded_descs

We flatten the corpus of words to get the most frequend terms as a guide for which lithology classes we can define.

In [ ]:
%%time
flat = flat_list_tokens(y)
len(set(flat))

In [ ]:
df_most_common= token_freq(flat, 50)

In [ ]:
df_most_common

In [ ]:
plot_freq(df_most_common)

In [ ]:
condamine_litho_cleaned_pkl = os.path.join(condamine_litho_dir, 'condamine_litho_cleaned.pkl')

if not os.path.exists(condamine_litho_cleaned_pkl):
    with open(condamine_litho_cleaned_pkl, 'wb') as handle:
        pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(condamine_litho_cleaned_pkl, 'rb') as handle:
        train_data = pickle.load(handle)

# Defining lithology classes

Starting with the three terms we want to end up with, we can also add the most frequent terms observed in the corpus.


In [ ]:
lithologies = ['alluvium', 'basalt', 'bedrock', 'clay', 'sandstone','sand','shale','soil','honeycomb','gravel','coal','gravel','silt','soil','rock', 'limestone', 'metal']
# see https://www.bioregionalassessments.gov.au/assessments/21-22-data-analysis-clarence-moreton-bioregion/21211-lithological-and-stratigraphic-data
# for 'metal' or 'blue metal'
any_litho_markers_re = r'alluvium|sand|clay|ston|shale|basa|silt|soil|honey|coal|gravel|rock|mud|metal'

regex = re.compile(any_litho_markers_re)

lithologies_dict = dict([(x,x) for x in lithologies])
lithologies_dict['sands'] = 'sand'
lithologies_dict['basalts'] = 'basalt'
lithologies_dict['clays'] = 'clay'
lithologies_dict['shales'] = 'shale'
lithologies_dict['claystone'] = 'clay'
lithologies_dict['siltstone'] = 'silt'
lithologies_dict['mudstone'] = 'silt' # ??
lithologies_dict['capstone'] = 'limestone' # ??
lithologies_dict['ironstone'] = 'sandstone' # ??
lithologies_dict['topsoil'] = 'soil' # ??

lithologies_adjective_dict = {
    'sandy' :  'sand',
    'clayey' :  'clay',
    'clayish' :  'clay',
    'shaley' :  'shale',
    'silty' :  'silt',
    'gravelly' :  'gravel'
}

In [ ]:
%%time
v_tokens = v_word_tokenize(y)

In [ ]:
%%time
vt = v_find_litho_markers(v_tokens, regex=regex)

In [ ]:
zero_mark = len([x for x in vt if len(x) == 0 ])
at_least_one_mark = len([x for x in vt if len(x) >= 1])
at_least_two_mark = len([x for x in vt if len(x) >= 2])
print('There are %s entries with no marker, %s entries with at least one, %s with at least two'%(zero_mark,at_least_one_mark,at_least_two_mark))

# Testing the regular expression model on the Condamine dataset

We have one first pass at classifying into one of the three target classes.

In [ ]:
prim_litho = [find_primary_lithology(x, lithologies_dict) for x in vt]

In [ ]:
n = len(set(prim_litho))
plot_freq(token_freq(prim_litho, n_most_common = n))

We define a map of classes down to alluvium, bedrock, basalt or unknown.

We will do a bit of "snooping" on the labelled data; this is not ideal but is necessary to validate some classification assumptions.

Given the a priori classification of primary lithologies, what proportions of labels do we have for each primary lithologhy?



In [ ]:
tmp_df = pd.DataFrame({ LITHO_CLASS_COL: train_data[LITHO_CLASS_COL], REGEX_LITHO_CLASS_COL: prim_litho, LITHO_DESC_COL: descs})

In [ ]:
def class_freq_for_lithology(litho_name):
    blah = tmp_df.loc[tmp_df[REGEX_LITHO_CLASS_COL] == litho_name]
    return token_freq(blah[LITHO_CLASS_COL].values, 50)

In [ ]:
class_freq_for_lithology('clay')

In [ ]:
class_freq_for_lithology('sand')

In [ ]:
class_freq_for_lithology('soil')

In [ ]:
class_freq_for_lithology('honeycomb')

In [ ]:
class_freq_for_lithology('shale')

In [ ]:
class_freq_for_lithology('coal')

In [ ]:
class_freq_for_lithology('gravel')

In [ ]:
class_freq_for_lithology('silt')

In [ ]:
lithology_map={
    'alluvium' :  'alluvium',
    'bedrock' :  'bedrock',
    'basalt' :  'basalt',
    'metal' :  'basalt',
    'honeycomb' :  'basalt',
    'clay' :  'alluvium',
    'coal' :  'bedrock', 
    'sandstone' :  'bedrock',
    'sand' :  'alluvium',
    '' :  'unknown',
    'soil' :  'alluvium',
    'shale': 'bedrock',
    'gravel': 'alluvium',
    'silt' : 'bedrock',
    'rock' : 'bedrock',
    'limestone' : 'alluvium'
}

In [ ]:
final_prim_litho=list()
for x in prim_litho:
    final_prim_litho.append(lithology_map[x])


In [ ]:
token_freq(final_prim_litho)

In [ ]:
simplified_lithology=train_data[LITHO_CLASS_COL].values

In [ ]:
def get_accuracy(modelled, expected):
    matches = np.equal(modelled, expected)
    return np.count_nonzero(matches)/len(matches)

In [ ]:
print("Accuracy of regex for classifying primary lithologies: ", get_accuracy(final_prim_litho, simplified_lithology))

Let's check what lithology descriptions led to an 'unknown' classification to see whether we are missing something

In [ ]:
tmp_df = pd.DataFrame({ LITHO_CLASS_COL: train_data[LITHO_CLASS_COL], REGEX_LITHO_CLASS_COL: final_prim_litho, LITHO_DESC_COL: descs})

In [ ]:
match_and_sample_df(tmp_df, 'unknown', colname=REGEX_LITHO_CLASS_COL, size=100)

In [ ]:
df_unk = tmp_df.loc[ tmp_df[REGEX_LITHO_CLASS_COL] == 'unknown' ]
df_unk.head()

In [ ]:
flat = flat_list_tokens(df_unk[LITHO_DESC_COL].values)

In [ ]:
s = ' '.join(flat)

In [ ]:
show_wordcloud(s, title = 'Unclassified via regexp')

In [ ]:
plot_freq(token_freq(flat, n_most_common = 30))

We did not use 'loam' as a term. There are others that should be used, but it is unclear how they should be remapped, like "metal" and "file"

In [ ]:
lithologies.append('loam')
lithologies.append('granite')
lithologies.append('soapstone')

In [ ]:
any_litho_markers_re = any_litho_markers_re + '|loam|granite|soap'
regex = re.compile(any_litho_markers_re)
lithologies_dict['loam'] = 'loam'
lithologies_dict['granite'] = 'granite'
lithologies_dict['soapstone'] = 'soapstone'

In [ ]:
v_tokens = v_word_tokenize(y)
vt = v_find_litho_markers(v_tokens, regex=regex)

In [ ]:
prim_litho = [find_primary_lithology(x, lithologies_dict) for x in vt]

In [ ]:
n = len(set(prim_litho))
plot_freq(token_freq(prim_litho, n_most_common = n))

In [ ]:
lithology_map['loam'] = 'alluvium'
lithology_map['granite'] = 'bedrock'
lithology_map['soapstone'] = 'bedrock'

In [ ]:
final_prim_litho=list()
for x in prim_litho:
    final_prim_litho.append(lithology_map[x])


In [ ]:
token_freq(final_prim_litho)

In [ ]:
tmp_df = pd.DataFrame({ LITHO_CLASS_COL: train_data[LITHO_CLASS_COL], REGEX_LITHO_CLASS_COL: final_prim_litho, LITHO_DESC_COL: descs})

In [ ]:
match_and_sample_df(tmp_df, 'unknown', colname=REGEX_LITHO_CLASS_COL, size=100)

In [ ]:
print("Accuracy of regex for classifying primary lithologies: ", get_accuracy(final_prim_litho, simplified_lithology))

In [ ]:
tmp_df = pd.DataFrame({ LITHO_CLASS_COL: train_data[LITHO_CLASS_COL], REGEX_LITHO_CLASS_COL: final_prim_litho, LITHO_DESC_COL: descs})

In [ ]:
df_unk = tmp_df.loc[ tmp_df[REGEX_LITHO_CLASS_COL] == 'unknown' ]
df_unk.head()

In [ ]:
flat = flat_list_tokens(df_unk[LITHO_DESC_COL].values)

In [ ]:
s = ' '.join(flat)

In [ ]:
show_wordcloud(s, title = 'Unclassified via regexp')

In [ ]:
def regex_matches(x, regex):
    return (regex.match(x) is not None)

def find_regex_df(df, expression, colname):
    """Sample a random subset of rows where the lithology column matches a particular class name.

        Args:
            df (pandas data frame): bore lithology data  with columns named PRIMARY_LITHO_COL
    
        Returns:
            dataframe:
    """
    tested = df[colname].values
    regex = re.compile(expression)
    xx = [(regex.match(x) is not None) for x in tested]
    df_test = df.loc[xx]
    return df_test

In [ ]:
# find_regex_df( df_unk, '.*basalt.*', LITHO_DESC_COL)
# find_regex_df( df_unk, '.*file.*', LITHO_DESC_COL)

match_and_sample_df(tmp_df, 'unknown', colname=REGEX_LITHO_CLASS_COL, out_colname=None, size=50, seed=0)

In [ ]:
df_regex = df.copy()
df_regex[REGEX_LITHO_CLASS_COL] = final_prim_litho

In [ ]:
condamine_litho_regex = os.path.join(condamine_litho_dir, 'condamine_litho_regex.pkl')
with open(condamine_litho_regex, 'wb') as handle:
    pickle.dump(df_regex, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Testing the deep learning model on the same dataset

In [ ]:
# train_data=train_data_unprocessed.copy()

In [ ]:
# conda install gensim
# conda install tensorflow
# conda install keras
# pip install wordcloud

from ela.experiment.textproc import Model

In [ ]:
train_data_dl = train_data.copy()
model=Model(train_data_dl,20)

In [ ]:
model.initialise_model()

As of 2019-07-20: 

* Training Accuracy: 0.8620
* Testing Accuracy:  0.8562


In [ ]:
x = train_data.copy()
x.head()

In [ ]:
x['Description'] = x['Lithology_original']
#model.predict(x)

In [ ]:
predictions_dl = pd.read_csv('prediction_file.csv')

In [ ]:
predictions_dl.head()

In [ ]:
print("Accuracy of DL for classifying primary lithologies: ", get_accuracy(predictions_dl['Simplified_Lithology'], simplified_lithology))

## Explaining the predictive performance of regexp versus DL

* Confusion matrices for RE and DL
* Subset each case where DL gets is right and not RE, and get a wordcloud of terms
* Subset each case where RE gets is right and not DL, and get a wordcloud of terms

In [ ]:
##

condamine_litho_regex = os.path.join(condamine_litho_dir, 'condamine_litho_regex.pkl')
with open(condamine_litho_regex, 'rb') as handle:
    df_regex = pickle.load(handle)

In [ ]:
from sklearn.metrics import confusion_matrix

df_regex.head()

In [ ]:
litho_class_names = ["basalt", "bedrock", "alluvium", "unknown"]

In [ ]:
def build_confusion_matrix(df_regex, colname_true, colname_predicted, labels = litho_class_names):
    y_true = df_regex[colname_true].values
    y_pred = df_regex[colname_predicted].values    
    m = confusion_matrix(y_true, y_pred, labels=labels)
    return m

def normalise_confusion_matrix(m):
    fractions = m / m.astype(np.float).sum(axis=0)
    return fractions

def plot_cf_matrix(m, litho_class_names, title='confusion matrix', figsize = (10,7), cmap='coolwarm', center=None):
    """
    """
    df_cm = pd.DataFrame(m, index = [i for i in litho_class_names],
                      columns = [i for i in litho_class_names])
    plt.figure(figsize = figsize)
    sn.heatmap(df_cm, annot=True, cmap=cmap, center=center)
    plt.title(title)
    plt.xlabel('predicted')
    plt.ylabel('actual')
    plt.show()
    

In [ ]:
m_regex = build_confusion_matrix(df_regex, colname_true=LITHO_CLASS_COL, colname_predicted=REGEX_LITHO_CLASS_COL)
fractions_regex = normalise_confusion_matrix(m_regex)

m_dl = build_confusion_matrix(predictions_dl, colname_true='Simplified_lithology', colname_predicted='Simplified_Lithology')
fractions_dl = normalise_confusion_matrix(m_dl)

In [ ]:
diff_frac = fractions_dl - fractions_regex

In [ ]:
plot_cf_matrix(m_regex, litho_class_names, title='confusion matrix')

In [ ]:
plot_cf_matrix(fractions_regex, litho_class_names)

In [ ]:
plot_cf_matrix(m_dl, litho_class_names)

In [ ]:
plot_cf_matrix(fractions_dl, litho_class_names)

In [ ]:
plot_cf_matrix(diff_frac, litho_class_names, title='dl-regex confusion matrix comparison', cmap="PiYG", center=0)

In [ ]:
plot_cf_matrix(diff_frac * 100, litho_class_names, title='dl-regex confusion matrix comparison', cmap='bwr', center=0)

In [ ]:
# double-checking the normalisation procedure
#true_counts = m.astype(np.float).sum(axis=1) # true preds
#f = m / true_counts
#f
#f = m / m.astype(np.float).sum(axis=0)
#f
#f.sum(axis=0), f.sum(axis=1)

# Checking the accuracy if geolocation and log descriptions are taken as input features 

Logistic Regression for classifying lithologies based on geolocation. Combining the outputs of the logistic regression model and the deep learning model.

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
train_data = train_data_unprocessed.copy()
set(train_data['Simplified_lithology'].values)

In [ ]:
train_data['Simplified_lithology']=train_data['Simplified_lithology'].replace(np.nan,'Unknown',regex=True)

In [ ]:
train_data['Simplified_lithology']=train_data['Simplified_lithology'].str.lower()

In [ ]:
set(train_data['Simplified_lithology'].values)

In [ ]:
train_data['Simplified_lithology'],labels=pd.factorize(train_data['Simplified_lithology'])

In [ ]:
train_X=train_data[['EASTING','NORTHING']][0:len(model.train_X)]
test_X=train_data[['EASTING','NORTHING']][len(model.train_X):]
train_y=train_data['Simplified_lithology'][0:len(model.train_X)]
test_y=train_data['Simplified_lithology'][len(model.train_X):]
train_X.replace(np.nan,0.0,inplace=True)
test_X.replace(np.nan,0.0,inplace=True)
print(train_X.shape)
print(train_y.shape)


In [ ]:
clf=LogisticRegression(C=0.01)

In [ ]:
clf.fit(train_X,train_y)

In [ ]:
y_pred=clf.predict_proba(test_X)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
train_data=train_data_unprocessed.copy()
new_df=pd.DataFrame(train_data[['Lithology_original']].values,columns=['Description'])

In [ ]:
new_df.head()

In [ ]:
new_df = new_df[len(model.train_X):].copy()

In [ ]:
y_pred_dl=model.predict_certainity(new_df)

In [ ]:
y_pred, y_pred_dl

In [ ]:
print(y_pred.shape)
print(y_pred_dl.shape)

In [ ]:
final_class_prob=np.mean(np.array([y_pred,y_pred_dl]), axis=0 )

In [ ]:
print(final_class_prob.shape)

In [ ]:
final_output_numerical=np.argmax(final_class_prob,axis=1)

In [ ]:
print(accuracy_score(test_y,final_output_numerical))

In [ ]:
simplified_lithology_categories=[]
for x in final_output_numerical:
    simplified_lithology_categories.append(labels[x])

It does not increase the accuracy. Don't think the geolocation has an impact on the simplified lithologies

# Ontology based learning

Python provides libraries such as RDFLib, OWL 2 for working with ontologies. OWL 2 is better suited for ontology oriented programming since it offers a more pythonic way of managing/creating ontologies.
But then, RDFLib works well with .rdf files. Hence, there is a trade-off. Can use based on individual project needs. <br>

Protege is an 'IDE' which can be used to manage/create ontologies. It was developed at Stanford and can be downloaded from here [Protege](https://protege.stanford.edu/)<br>
Protege can be used to visualise the different relationships defined in an ontology <br>

The ontology that I am using can be downloaded from [here](http://ontologydesignpatterns.org/wiki/Ontology:CGI_Simple_Lithology_201001)

In [ ]:
import rdflib as rd

In [ ]:
from rdflib.namespace import SKOS
from rdflib.namespace import RDFS

In [ ]:
ontology=rd.Graph()
condamine_litho_ontology = os.path.join(condamine_litho_dir, 'SimpleLithology201001.rdf')
ontology.parse(condamine_litho_ontology)

In [ ]:
lithology_dictionary=dict()
for x,y in ontology.subject_objects(SKOS.prefLabel):
    lithology_dictionary[x]=y
    

In [ ]:
for x,y in ontology.subject_objects(SKOS.broader):
    if x in lithology_dictionary.keys() and y in lithology_dictionary.keys():
        print(" broader class of ",lithology_dictionary[x]," is ",lithology_dictionary[y])

<h5>These relationships can be converted into fuzzy if-then rules and fed to the machine learning model to make better decisions. <br>
Above is only one kind of relationship. We have other relationships such as narrower, description of each label etc. All these combined would make a very "knowledgeable" machine learning model. </h5>

### Comparing visually

Optional



## Observations and discussions

DL much better than regular expressions.<br>

Regular expressions use a set of keywords and look for matches in the descriptions. Sometimes, these keywords might not be in the descriptions.<br>

Similarly, regular expressions map descriptions to a set of predefined catgories (clay,sand,etc.) which are further refined into broader categories ( alluvium,basalt,bedrock). Geoscientists confirm that the mapping is not one/many-to-one. For example,Clay could be part of alluvium/basalt. This cannot be achieved using the regular expression model.

DL, on the other hand, learns the descriptions of different lithology classes. Based on labelled data from geoscientists, DL model learns the different kinds of descriptions which would pertain to for e.g alluvium. 




## Conclusion and future work